In [808]:
import os 
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
import panel as pn
import tempfile
import copy
import random
import io
import json
from langchain.vectorstores import Chroma
from bokeh.sampledata.autompg import autompg
from io import StringIO
import pandas as pd
from math import pi

from bokeh.palettes import Pastel1
from bokeh.plotting import figure
from bokeh.transform import cumsum
import pandas as pd
from evaluate import load
        

In [809]:
import os
from os.path import join
import json
import openai
from IPython.display import display, Markdown

# chatOpenAI within langchain
from langchain.chat_models import ChatOpenAI

# prompt template refers to a reproducible way to generate a prompt
from langchain.prompts import ChatPromptTemplate

# LangChain for question answering over a list of documents
from langchain.chains import RetrievalQA

# Load data from a source as Document's. A Document is a piece of text and associated metadata.
from langchain.document_loaders import CSVLoader
from langchain.document_loaders import PyPDFLoader

# DocArrayInMemorySearch is a document index provided by Docarray that stores documents in memory
from langchain.vectorstores import DocArrayInMemorySearch


#takes care of storing embedded data and performing vector search for you
from langchain.indexes import VectorstoreIndexCreator

# QA evaluator
from langchain.evaluation.qa import QAGenerateChain
# QA evaluates two responses
from langchain.evaluation.qa import QAEvalChain
# Load embeddings
from openai.api_resources.embedding import Embedding

# Load OpenAI Embeddings
from langchain.embeddings.openai import OpenAIEmbeddings

#Load LLMs
from langchain.llms import AzureOpenAI
from langchain.chat_models import AzureChatOpenAI

import os
from os.path import join
import json
import openai
from IPython.display import display, Markdown

# chatOpenAI within langchain
from langchain.chat_models import ChatOpenAI
from langchain.chat_models import AzureChatOpenAI

# prompt template refers to a reproducible way to generate a prompt
from langchain.prompts import ChatPromptTemplate

# LangChain for question answering over a list of documents
from langchain.chains import RetrievalQA

# Load data from a source as Document's. A Document is a piece of text and associated metadata.
from langchain.document_loaders import CSVLoader

# DocArrayInMemorySearch is a document index provided by Docarray that stores documents in memory
from langchain.vectorstores import DocArrayInMemorySearch


#takes care of storing embedded data and performing vector search for you
from langchain.indexes import VectorstoreIndexCreator

#pdf loader
from langchain.document_loaders import PyPDFLoader

# QA evaluator
from langchain.evaluation.qa import QAGenerateChain
from langchain.prompts import PromptTemplate
from langchain.chains.question_answering import load_qa_chain
from langchain.text_splitter import CharacterTextSplitter

from langchain.chains import SequentialChain
from langchain.chains import SimpleSequentialChain

from langchain.embeddings.openai import OpenAIEmbeddings

from langchain.memory import ConversationSummaryMemory

from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

In [810]:
CURRENT_PATH = os.getcwd()
CONFIG_FILE = "config.json"

# Load Config File
with open(join(CURRENT_PATH, CONFIG_FILE)) as file:
    # Load the JSON data
    config = json.load(file)

In [811]:
CHAT_NAME = 'Aurora'
# Environmental Variables
AZURE_OPENAI_KEY = config['AZURE_OPENAI_KEY']
AZURE_OPENAI_ENDPOINT = config['AZURE_OPENAI_ENDPOINT']
AZURE_ENGINE_NAME = config['AZURE_ENGINE_NAME']
AZURE_ADA_NAME = config['AZURE_ADA_NAME']
AZURE_ADA_ENDPOINT = config['AZURE_ADA_ENDPOINT']
AZURE_ADA_KEY = config['AZURE_ADA_KEY']

In [812]:
# OpenAI ADA API embeddings 
openai_ada_key = AZURE_ADA_KEY
openai_ada_base = AZURE_ADA_ENDPOINT
openai_ada_deployment = AZURE_ADA_NAME
openai_api_type = 'azure'


In [813]:
def textwrap(s):
    new_s = ""
    words = s.split(" ")
    for i in range(len(words)):
        new_s += " " + words[i]
        if i % 8 == 0 and i >0:
            new_s += '\n'
    return new_s.strip().replace("\n ", "\n")

In [814]:
import json
from pathlib import Path
from typing import Callable, Dict, List, Optional, Union

from langchain.docstore.document import Document
from langchain.document_loaders.base import BaseLoader

class JSONLoader(BaseLoader):
    def __init__(
        self,
        file_path: Union[str, Path],
        content_key: Optional[str] = None,
        ):
        self.file_path = Path(file_path).resolve()
        self._content_key = content_key
        
    def load(self) -> List[Document]:
        """Load and return documents from the JSON file."""

        docs=[]
        # Load JSON file
        with open(self.file_path) as file:
            data = json.load(file)

            # Iterate through 'pages'
            for page in data['employees']:
                name = page['name']
                title = page['title']
                parent = page['parent']
                department = page['department']
                key = page['key']
                ask = page['ask me about']

                text = ask
                metadata = dict(
                    name=name,
                    title=title,
                    parent=parent,
                    department=department,
                    key=key,
                    ask = ask)

                docs.append(Document(page_content=text, metadata=metadata))
        return docs


In [815]:
def save_dict(state, key=(), depth=0, max_depth=None, save_path='', load_path=None):
    filename_dict = {}
    for k, v in state.items():
        curkey = key+(k,)
        if depth < max_depth:
            filename_dict[k] = save_dict(v, curkey, depth+1, max_depth,
                                         save_path, load_path)
        else:
            filename = '_'.join([str(i) for i in curkey]) +'.json'
            filepath = os.path.join(save_path, filename)
            directory = os.path.dirname(filepath)
            if not os.path.exists(directory):
                os.makedirs(directory)
            with open(filepath, 'w') as f:
                json.dump(v, f)
            refpath = filepath
            if load_path:
                refpath = os.path.join(load_path, filename)
            filename_dict[k] = refpath
    return filename_dict

In [816]:
pbar_hb = 0
pbar_role = 0 
pbar_chart = 0
chart_hb = 0
chart_role = 0
chart_chart = 0

In [817]:
pn.extension('texteditor','floatpanel', template="fast", sizing_mode='stretch_width', notifications=True)


NotificationArea(notifications=[Notification(_destroyed=F...], sizing_mode='stretch_width')

# Generate Q&A

In [818]:
def check_char():
    if not os.path.exists('temp_files\\Organizational Chart.json'):
        return True
    else:
        return False

In [819]:
radio_group = pn.widgets.RadioButtonGroup(
    name='Radio Button Group', options=['Handbook', 'Organizational Chart',  'Role Documentation'], button_type='success', width = 400)


role_qa = pn.widgets.Button(name="Aurora, generate role specific Q&A", height=40, button_type='primary', disabled = (check_char()), width=300)



In [820]:

# Button generates Q&A, validates Q&A, saves Q&A
menu_items = ['Faster Q&A', 'More Accurate Q&A']
method_radio = pn.widgets.RadioButtonGroup(
    name='Radio Button Group', options=menu_items, button_type='warning', orientation= 'vertical', height = 90, width = 200)



In [821]:
def qa_prompt():
    from langchain.output_parsers.regex import RegexParser

    template = """You are a teacher coming up with questions to ask on a quiz about an organizational chart. 
    
    Given the following document, please generate a question and answer based on that document.
    Example Format:
    <Begin Document>
    ...
    <End Document>
    QUESTION: question here
    ANSWER: answer here
    
    These questions should be detailed and be based explicitly on information in the document.
    You never ask about what's the role of a specific person, for instance, you never ask "what is <person's name> role within the <company name>?"
    Instead, you ask specific questions about people responsibilities and what they want to be asked about.
    Begin!
    
    <Begin Document>
    {doc}
    <End Document>"""
    output_parser = RegexParser(
        regex=r"QUESTION: (.*?)\n+ANSWER: (.*)", output_keys=["query", "answer"]
    )
    PROMPT = PromptTemplate(
        input_variables=["doc"], template=template, output_parser=output_parser
    )
    return PROMPT

In [822]:
def qa_prompt_handbook():
    from langchain.output_parsers.regex import RegexParser

    template = """You are a teacher coming up with questions to ask on a quiz about a company's handbook. 
    
    Given the following document, please generate a question and answer based on that document.
    Example Format:
    <Begin Document>
    ...
    <End Document>
    QUESTION: question here
    ANSWER: answer here
    
    These questions should be detailed and be based explicitly on information in the document.
    You never ask about the metadata of the document, for instance, you never ask about the page number. 
    Begin!
    
    <Begin Document>
    {doc}
    <End Document>"""
    output_parser = RegexParser(
        regex=r"QUESTION: (.*?)\n+ANSWER: (.*)", output_keys=["query", "answer"]
    )
    PROMPT = PromptTemplate(
        input_variables=["doc"], template=template, output_parser=output_parser
    )
    return PROMPT

In [823]:
from langchain.vectorstores import Chroma
question_db = []
questions = []
vectordb_dict={}
def qa_output(_):
    global questions
    global vectordb_dict
    question_db = []
    # load PDF File
    if file_input.value is not None:
        embedding = OpenAIEmbeddings(openai_api_key=openai_ada_key
                                      , openai_api_base = openai_ada_base
                                      , deployment = openai_ada_deployment
                                      , openai_api_type = openai_api_type
                                      , chunk_size = 1)
        
        llm = AzureChatOpenAI(openai_api_base=AZURE_OPENAI_ENDPOINT
                 ,openai_api_key=AZURE_OPENAI_KEY
                 ,openai_api_type='azure'
                 ,openai_api_version="2023-05-15"
                 ,deployment_name =AZURE_ENGINE_NAME
                 ,model = "gpt-3.5-turbo"
                 ,temperature = 0)      
        
        if '.pdf' in file_input.filename: 
            question_db = []
            file_input.save(f"temp_files\\{radio_group.value}.pdf")
    
            loader = PyPDFLoader(f"temp_files\\{radio_group.value}.pdf")
            data = loader.load()
            
            text_splitter= CharacterTextSplitter(
                separator='\n',
                chunk_size=1000,
                chunk_overlap=150,
                length_function=len
            )
            data_db=text_splitter.split_documents(data)
            vectordb=Chroma.from_documents(
                documents=data_db,
                embedding=embedding
            )
            print('pdf load completed')
            example_gen_chain = QAGenerateChain(prompt=qa_prompt_handbook(), llm = llm)



        elif '.json' in file_input.filename:
            question_db = []
            file_input.save(f"temp_files\\{radio_group.value}.json")
            loader = JSONLoader(file_path=f"temp_files\\{radio_group.value}.json")
            data = loader.load()
            role_qa.disabled = False
            data_db=json.load(open(f"temp_files\\{radio_group.value}.json"))
            data_db=json.dumps(data_db).split("key")
            vectordb=Chroma.from_texts(
                texts=data_db,
                embedding=embedding
            )
            print('json load completed')

            example_gen_chain = QAGenerateChain(prompt=qa_prompt(), llm = llm)
        
        
        #index = VectorstoreIndexCreator(vectorstore_cls=DocArrayInMemorySearch
        #                                , embedding= embedding ).from_loaders([loader])
        
        ###create vectordb for buddy
        vectordb_dict[radio_group.value]=vectordb
        print('saved vector')
        print(vectordb_dict)
        ###
    
        questions = example_gen_chain.apply_and_parse([{"doc": t} for t in data])
        
        for question in questions:
            question['answer'] = textwrap(question['answer']) 
            question['query'] = textwrap(question['query']) 
        
        question_db.extend([pn.pane.Markdown(f"A total of **{str(len(questions))}** questions were generated. \nHere's a sample of questions:")
                            , pn.Row(random.sample(questions, 5) )])

        if method_radio.value == 'Faster Q&A':
            pn.state.notifications.success(f"""Aurora has generated the questions, as requested""", duration=8000)
            save_qa_questions('1')
            return pn.Column(*question_db, margin=15, width=1100, min_height=200)
        else:
            pn.state.notifications.success(f"""Aurora has generated questions and is now evaluating them.""", duration=8000)
            
            predictions = qa_evaluate('1')
            print(predictions)
            pn.state.notifications.success(f"""Aurora has evaluated the questions.""", duration=8000)
            corrections = []
            for pred in predictions:
                if pred['evaluation'] == 'INCORRECT':
                    corr = {}
                    corr['query'] = pred['query']
                    corr['wrong answer'] = pred['answer']
                    corr['corrected answer'] = pred['result']
                    corrections.append(corr)
            if len(corrections)>0:
                question_db.extend([pn.Row(pn.pane.Markdown(f"A total of **{str(len(corrections))}** corrections were made. \nHere's a sample of those corrections:"),
                                           pn.Row(corrections), width=600 )])
            else:
                question_db.extend([pn.pane.Markdown(f"No corrections were needed!")])
            accept_new_answers('1')
            save_qa_questions('1')
            return pn.Column(*question_db, margin=15, width=1100, min_height=200)    
    return pn.Column(*question_db, margin=15,  min_height=200)
    #width_policy = 'fit' , width=1100,

In [824]:
def role_specific_questions(role):
    
    ai=AzureChatOpenAI(openai_api_base=AZURE_OPENAI_ENDPOINT
             ,openai_api_key=AZURE_OPENAI_KEY
             ,openai_api_type='azure'
             ,openai_api_version="2023-05-15"
             ,deployment_name =AZURE_ENGINE_NAME
             ,model = "gpt-3.5-turbo"
             ,temperature=0.1
        )
    
    convo=ConversationChain(llm=ai, verbose=False, memory=ConversationBufferMemory())
    intro="Give me 15 simple, theoretical questions+answers to evaluate knowledge aimed for someone with the role of "
    format='; Return the answer in the following format: [{"query": "insert question here", "answer": "insert answer here."}, {"query": "insert question here", "answer": "insert answer here."},...]'
    convo.predict(input=intro+role+format)
    questions=convo.memory.chat_memory.messages[-1].content
    questions=questions[questions.find('[{'):questions.rfind('}]')+len('}]')]
    
    return questions

In [825]:
question_db = []
questions = []
def qa_role_specific(_):
    global questions
    question_db = []
    # load PDF File
    #role_prompt.value
    print(role_prompt.value)
    if (role_prompt.value is not None) & (role_prompt.value!= ''):
        role = role_prompt.value
        questions = role_specific_questions(role)
        role_prompt.value_input = ''
        role_prompt.value = ''
        print(questions)
        radio_group.value = 'Role Documentation'

        sample_questions = random.sample(eval(questions), 5)
        with open(f'qa\\qa_{radio_group.value}.json', 'w') as f:         
            f.write(str(questions))        

        for question in sample_questions:
            question['answer'] = textwrap(question['answer']) 
            question['query'] = textwrap(question['query']) 

        pn.state.notifications.success(f"""Aurora has saved you specific questions for the role {role}""", duration=8000)
        question_db.extend([pn.pane.Markdown(f"A total of **{str(len(eval(questions)))}** questions were generated. \nHere's a sample of questions:")
                            , pn.Row(sample_questions, width=600 )])
        print(question_db)
    return pn.Column(*question_db, margin=15, width=1100, min_height=200)

In [826]:
# QA validation
evaluate_db =[]
predictions = []
def qa_evaluate(_): 
    global predictions
    global evaluate_db
    print(len(questions))
    if len(evaluate_db)>0:
        evaluate_db = evaluate_db[:-1]
    if len(questions) >0:
        if '.pdf' in file_input.filename: 
            loader = PyPDFLoader(f"temp_files\\{radio_group.value}.pdf")
            data = loader.load()
        elif '.json' in file_input.filename:
            loader = JSONLoader(f"temp_files\\{radio_group.value}.json")
            data = loader.load()
            role_qa.disabled = False
    
        embedding = OpenAIEmbeddings(openai_api_key=openai_ada_key
                              , openai_api_base = openai_ada_base
                              , deployment = openai_ada_deployment
                              , openai_api_type = openai_api_type
                              , chunk_size = 1)
    
    # llm initiation
        llm = AzureChatOpenAI(openai_api_base=AZURE_OPENAI_ENDPOINT
                 ,openai_api_key=AZURE_OPENAI_KEY
                 ,openai_api_type='azure'
                 ,openai_api_version="2023-05-15"
                 ,deployment_name =AZURE_ENGINE_NAME
                 ,model = "gpt-3.5-turbo"
               )
    
        index = VectorstoreIndexCreator(vectorstore_cls=DocArrayInMemorySearch
                                        , embedding= embedding ).from_loaders([loader])
    
        qa = RetrievalQA.from_chain_type(llm,retriever=index.vectorstore.as_retriever(), chain_type="stuff"
                                       , verbose=True
                                       , chain_type_kwargs = {"document_separator": "<<<<>>>>>"} )
        predictions = qa.apply(questions)
        
        eval_chain = QAEvalChain.from_llm(llm)
        graded_outputs = eval_chain.evaluate(questions, predictions)

        
        for question in predictions:
            question['answer'] = textwrap(question['answer']) 
            question['query'] = textwrap(question['query']) 
            question['result'] = textwrap(question['result']) 
            question['evaluation'] = graded_outputs[predictions.index(question)]['results']
        
#        evaluate_db.extend([pn.Row(predictions, width=600 )
#                           ,pn.Row(accept_evaluation)])          
    return predictions
    


In [827]:
def accept_new_answers(_):
    global predictions
    update_text = ""
    for i in range(len(questions)):
        if predictions[i]['evaluation'] != 'CORRECT':
            questions[i]['answer'] = predictions[i]['result']
    if len(predictions)>0:
        pn.state.notifications.info(f"""Q&A was updated""", duration=8000)
    return pn.pane.Markdown(update_text)

In [828]:
def save_qa_questions(_):
    saved_text = ""
    if 'questions' in globals(): 
        if len(questions)>0:
            with open(f'qa\\qa_{radio_group.value}.json', 'w') as f:         
                f.write(str(questions))
    if not os.path.exists(f'qa\\qa_{radio_group.value}.json'):
        saved_text = ""
    elif os.path.getsize(f'qa\\qa_{radio_group.value}.json') == 0:
        saved_text = ""
    else:
        pn.state.notifications.info(message = """File was saved!""", duration=8000)          
    return pn.pane.Markdown(saved_text)

In [829]:
introduction_qa = pn.Row(pn.pane.Alert(f"""{CHAT_NAME} will generate questions from your company’s existing documentation"""))

#file input
file_input = pn.widgets.FileInput(width=300, height=40)

run_qa = pn.widgets.Button(name="Aurora, generate Q&A from file!", styles=dict(background='White', text = 'white'), width=300, height =40)

validate_qa = pn.widgets.Button(name="Aurora, are these answers correct?", styles=dict(background='White', text = 'white'), width=300, height =50)

save_qa = pn.widgets.Button(name="Save my Q&A", styles=dict(background='White', text = 'white'), width=300, height =50)



accept_evaluation = pn.widgets.Button(name="Accept new answers", styles=dict(background='White', text = 'white'), width=150, height =50)

role_prompt = pn.widgets.TextInput(value="", placeholder='Enter role here...',  styles={'width': '300px', 'height': '50px'})

# File Input + Button
tooltip_qa = pn.widgets.TooltipIcon(value="Button will only become available once an Organizational Chart has been uploaded", position="right", width = 10 )

generate_qa_input = pn.Column(file_input, run_qa, width = 310)
methods = pn.Column(method_radio,  width = 210)
role_Column = pn.Column(pn.Row(role_qa, tooltip_qa, width = 300), role_prompt,  width = 710)
generate_qa = pn.Row(generate_qa_input, methods, role_Column,width_policy='fixed', width=1000)
#methods = pn.Row(run_qa, role_prompt)
# Validate & Save
qa_val_save = pn.Row(validate_qa, save_qa)


# Q&A Page
generate_qa_interactive = pn.panel(pn.bind(qa_output,run_qa), loading_indicator=True)

# Q&A Validate Asnwers page
#qa_validate_interactive = pn.panel(pn.bind(qa_evaluate, validate_qa), loading_indicator=True)

# Accept Answers
qa_accept_interactive = pn.panel(pn.bind(accept_new_answers, accept_evaluation), loading_indicator=True)

# Save Answers
qa_save_interactive = pn.panel(pn.bind(save_qa_questions, save_qa), loading_indicator=True)

# Save Answers
qa_role_interactive = pn.panel(pn.bind(qa_role_specific, role_qa), loading_indicator=True)

# Q&A Output
output_qa = pn.WidgetBox(generate_qa_interactive
                         , qa_role_interactive
                         , width=1100, scroll=True)





# Evaluate

In [830]:
all_questions = {}
categories = []
def load_questions():
    global all_questions
    global chat_text
    out = pn.Row()
    if len(all_questions)>0:
        out = pn.Row(pn.pane.Markdown( "Aurora:", width=100)
           ,pn.pane.Markdown( 'Let''s begin checking your knowledge!', width=600)) 
    for file_name in os.listdir(join(CURRENT_PATH, 'qa')):
        file_path = os.path.join(join(CURRENT_PATH, 'qa'), file_name)
        if os.path.isfile(file_path):
            with open(file_path, "r") as file:
                file_contents = file.read()
                # Remove newline characters and append the contents to the concatenated list
            if 'Handbook' in file_path:
                all_questions['Handbook'] = eval(file_contents)               
            elif 'Organizational Chart' in file_path:
                all_questions['Organizational Chart'] = eval(file_contents)              
            elif 'Role Documentation' in file_path: 
                all_questions['Role Documentation'] = eval(file_contents)
    return out
start_evaluation =pn.widgets.Button(name="Start Quiz", button_type='primary',  width = 100)


In [831]:
ne_prompt = pn.widgets.TextInput(value="", placeholder='Enter text here...', height = 50)
button_conversation = pn.widgets.Button(name="Click here to start the quiz!", width = 200, height = 40)


In [832]:
context = []
panels = [] 
counter = 0
qa_aurora = {}
ne_answer = []
my_questions = []
cyk_hist = []

In [833]:
def qa_evaluator(_):
    global all_questions
    global context
    global counter
    global qa_aurora
    global ne_answer
    global chart_chart
    global chart_hb
    global chart_role
    global pbar_chart
    global pbar_hb
    global pbar_role
    global my_questions
    global categories
    global cyk_hist
    
    llm = AzureChatOpenAI(openai_api_base=AZURE_OPENAI_ENDPOINT
             ,openai_api_key=AZURE_OPENAI_KEY
             ,openai_api_type='azure'
             ,openai_api_version="2023-05-15"
             ,deployment_name =AZURE_ENGINE_NAME
             ,model = "gpt-3.5-turbo"
           )
    counter += 1
    #Get the value introduced by the New Employee
    prompt = ne_prompt.value
    ne_prompt.value = ''
    if counter==2: 
        button_conversation.name = 'Send Input'
        load_questions()

        # generate questions and/or update questions
        if len(my_questions) == 0:
            my_questions.append(random.sample(all_questions['Handbook'], 3 - int(progress_handbook.value / 2 )))            
            my_questions.append(random.sample(all_questions['Role Documentation'], 3 - int(progress_role.value / 2 )))            
            my_questions.append(random.sample(all_questions['Organizational Chart'], 3 - int(progress_chart.value / 2 )))         

            categories.extend(['Handbook', 'Handbook', 'Handbook'
                               , 'Role Documentation', 'Role Documentation', 'Role Documentation'
                              , 'Organizational Chart','Organizational Chart', 'Organizational Chart'])
            my_questions = [item for sublist in my_questions for item in sublist]

            
        else:
            n_questions = 9 - int(progress_chart.value / 2) - int(progress_handbook.value / 2 )- int(progress_role.value / 2 )
            if len(my_questions) > n_questions:
                my_questions.pop(0)
                categories.pop(0)
            else:
                my_questions.append(random.sample(all_questions[categories[0]],1))
                categories.append(categories[0])
                my_questions.pop(0)
                categories.pop(0)

        
        if len(my_questions)>0:
            qa_aurora = my_questions[0]
            question_aurora = qa_aurora['query']

            # update chart
            if categories[0] == 'Organizational Chart':
                chart_chart += 1
            elif categories[0] == 'Handbook':
                chart_hb += 1
            elif categories[0] == 'Role Documentation':
                chart_role += 1
                
            panels.append(pn.Row(pn.pane.Markdown( "Aurora:", width=100),pn.pane.Markdown(question_aurora, width=600)))
            
    elif (counter > 2):   
        
        # generate questions and/or update questions

        eval_chain = QAEvalChain.from_llm(llm)
        ne_answer = {}
        ne_answer['query'] = qa_aurora['query']
        ne_answer['answer'] = qa_aurora['answer']
        ne_answer['result'] = prompt

        print('teste1')

        graded_outputs_2 = eval_chain.evaluate([qa_aurora], [ne_answer])

        ne_answer_hist = {}
        ne_answer_hist['query'] = qa_aurora['query']
        ne_answer_hist['answer'] = qa_aurora['answer']
        ne_answer_hist['result'] = prompt
        ne_answer_hist['eval'] = graded_outputs_2[0]['results']
        
        cyk_hist.append(ne_answer_hist)
        
        print('teste2')
        # Evaluate with SQuaD
        eval_ne_answers = [ne_answer]
        print([ne_answer])
        for i, eg in enumerate(eval_ne_answers):
            answers = eg.copy()
            eval_ne_answers[i]["id"] = str(i)
            eval_ne_answers[i]["prediction_text"] = eval_ne_answers[i]["answer"]
            eval_ne_answers[i]['no_answer_probability'] = 0
            del eval_ne_answers[i]["answer"]
            del eval_ne_answers[i]["query"]
            del eval_ne_answers[i]["result"]
        
            answers["id"] = str(i)
            answers["answers"] = {"text": [answers["result"]], "answer_start": [0]}
            del answers["query"]
            del answers['result']
            del answers['answer']     
            
        #check if answer == question
        eval_q_eq_a = [ne_answer]
        print([ne_answer])
        for i, eg in enumerate(eval_q_eq_a):
            validate_q_eq_a = eg.copy()
            eval_q_eq_a[i]["id"] = str(i)
            eval_q_eq_a[i]["prediction_text"] = eval_q_eq_a[i]["query"]
            eval_q_eq_a[i]['no_answer_probability'] = 0
            del eval_q_eq_a[i]["answer"]
            del eval_q_eq_a[i]["query"]
            del eval_q_eq_a[i]["result"]
        
            validate_q_eq_a["id"] = str(i)
            validate_q_eq_a["answers"] = {"text": [validate_q_eq_a["result"]], "answer_start": [0]}
            del validate_q_eq_a["query"]
            del validate_q_eq_a['result']
            del validate_q_eq_a['answer']     
            

        
        squad_metric = load("squad_v2")
        eval_out = squad_metric.compute(references=[answers],predictions=[eval_ne_answers[0]])
        debug_q_eq_a = squad_metric.compute(references=[validate_q_eq_a],predictions=[eval_q_eq_a[0]])

        eval_agg = ('INCORRECT' if (debug_q_eq_a['f1']> 95) 
                    else 'CORRECT' if (graded_outputs_2[0]['results'] == 'CORRECT') & (eval_out['f1']> 20) 
                    else 'PARTIALLY CORRECT' if (graded_outputs_2[0]['results'] == 'PARTIALLY CORRECT') & (eval_out['f1']> 20)
                    else 'INCORRECT')

        # Update progress bars
        if debug_q_eq_a['f1']> 95:
            continue
        else:
            if (graded_outputs_2[0]['results'] == 'CORRECT') & (eval_out['f1']> 20):
                if categories[0] == 'Organizational Chart':
                    progress_chart.value += 2
                elif categories[0] == 'Handbook':
                    progress_handbook.value += 2
                elif categories[0] == 'Role Documentation':
                    progress_role.value += 2
            elif (graded_outputs_2[0]['results'] == 'PARTIALLY CORRECT') & (eval_out['f1']> 20):
                if categories[0] == 'Organizational Chart':
                    progress_chart.value += 1
                elif categories[0] == 'Handbook':
                    progress_handbook.value += 1
                elif categories[0] == 'Role Documentation':
                    progress_role.value  += 1


        if len(my_questions) == 0:
            my_questions.append(random.sample(all_questions['Handbook'], 3 - int(progress_handbook.value / 2 )))            
            my_questions.append(random.sample(all_questions['Role Documentation'], 3 - int(progress_role.value / 2 )))            
            my_questions.append(random.sample(all_questions['Organizational Chart'], 3 - int(progress_chart.value / 2 )))         

            categories.extend(['Handbook', 'Handbook', 'Handbook'
                               , 'Role Documentation', 'Role Documentation', 'Role Documentation'
                              , 'Organizational Chart','Organizational Chart', 'Organizational Chart'])
            my_questions = [item for sublist in my_questions for item in sublist]

        else:
            n_questions = 9 - int(progress_chart.value / 2) - int(progress_handbook.value / 2 )- int(progress_role.value / 2 )
            
            if len(my_questions) > n_questions:
                my_questions.pop(0)
                categories.pop(0)
            else:
                my_questions.append(random.sample(all_questions[categories[0]],1)[0])
                categories.append(categories[0])
                my_questions.pop(0)
                categories.pop(0)
        
        # update chart
        if len(categories)>0: 
            if categories[0] == 'Organizational Chart':
                chart_chart += 1
            elif categories[0] == 'Handbook':
                chart_hb += 1
            elif categories[0] == 'Role Documentation':
                chart_role += 1
                
            panels.append(
                pn.Row(pn.pane.Markdown( "New Employee:", width=100), pn.pane.Markdown(prompt, width=600)))
            panels.append(
                pn.Row(pn.pane.Markdown( "Aurora:", width=100), pn.pane.Markdown(f"The correct answer is: {qa_aurora['answer']}", width=600)))
            panels.append(
                pn.Row(pn.pane.Markdown( "Aurora:", width=100), pn.pane.Markdown(f"You are {eval_agg}", width=600)))
            
            print(my_questions)        
            qa_aurora = my_questions[0]
            print('\n\n qa_aurora: ' + str(qa_aurora))
            question_aurora = qa_aurora['query']
            panels.append(
                pn.Row(pn.pane.Markdown( "Aurora:", width=100)
               ,pn.pane.Markdown(question_aurora, width=600)))

        else:
                
            panels.append(
                pn.Row(pn.pane.Markdown( "New Employee:", width=100), pn.pane.Markdown(prompt, width=600)))
            panels.append(
                pn.Row(pn.pane.Markdown( "Aurora:", width=100), pn.pane.Markdown(f"The correct answer is: {qa_aurora['answer']}", width=600)))
            panels.append(
                pn.Row(pn.pane.Markdown( "Aurora:", width=100), pn.pane.Markdown(f"You are {eval_agg}", width=600)))            
            panels.append(pn.state.notifications.success('Congratulations! You''ve completed your Onboarding!.', duration=8000))

    return pn.Column(*panels, height = 300, scroll=True)

In [834]:
##cyk_hist
def cyk_recap(cyk_hist):
    ai=AzureChatOpenAI(openai_api_base=AZURE_OPENAI_ENDPOINT
             ,openai_api_key=AZURE_OPENAI_KEY
             ,openai_api_type='azure'
             ,openai_api_version="2023-05-15"
             ,deployment_name =AZURE_ENGINE_NAME
             ,model = "gpt-3.5-turbo"
             ,temperature=0.1
        )
    convo=ConversationChain(llm=ai, verbose=False)#, memory=ConversationBufferMemory())
    intro="We have a recorded conversation in the format [{'query': 'question 1', 'answer': 'theoretical answer 1', 'result': 'human answer 1', 'eval': 'evaluation 1'}, {'query': 'question 2', 'answer': 'theoretical answer 2', 'result': 'human answer 2', 'eval':'evaluation 2'}, ...], where 'query' is a question, 'answer' is a correct answer, 'result' is the human's answer and 'eval' indicates whether or not the human answer was correct or incorrect."
    request="I want you to summarize the conversation's learnings. Consider only the information that 'query' and 'answer' provides. Use terms such as 'we learned'. Don't use the term 'conversation'. Refer to the human in the second person. Conversation:"
    recap=convo.predict(input=intro+request+str(cyk_hist))
    return recap

from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

def cyk_weak(cyk_hist):
    ai=AzureChatOpenAI(openai_api_base=AZURE_OPENAI_ENDPOINT
             ,openai_api_key=AZURE_OPENAI_KEY
             ,openai_api_type='azure'
             ,openai_api_version="2023-05-15"
             ,deployment_name =AZURE_ENGINE_NAME
             ,model = "gpt-3.5-turbo"
             ,temperature=0.1
        )

    
    template="We have a recorded conversation in the format [{'query': 'question 1', 'answer': 'theoretical answer 1', 'result': 'human answer 1', 'eval': 'evaluation 1'}, {'query': 'question 2', 'answer': 'theoretical answer 2', 'result': 'human answer 2', 'eval':'evaluation 2'}, ...], where 'query' is a question, 'answer' is a correct answer, 'result' is the human's answer and 'eval' indicates whether or not the human answer was correct or incorrect. If there is no 'INCORRECT' in the conversation, return 'Congratulations! You're an expert'. Otherwise, I want you to explain to the human, refering to them in the second person, what are their main difficulties, based on their incorrect answers; don't give any concrete examples."
    system_message_prompt = SystemMessagePromptTemplate.from_template(template,template_format="jinja2")
    human_template="Conversation: {convo}"
    human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
    chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

    result=ai(chat_prompt.format_prompt(convo=str(cyk_hist)).to_messages())
    improve=result.content
    #convo=ConversationChain(llm=ai, verbose=False)#, memory=ConversationBufferMemory())
    #intro="We have a recorded conversation in the format [{'query': 'question 1', 'answer': 'theoretical answer 1', 'result': 'human answer 1', 'eval': 'evaluation 1'}, {'query': 'question 2', 'answer': 'theoretical answer 2', 'result': 'human answer 2', 'eval':'evaluation 2'}, ...], where 'query' is a question, 'answer' is a correct answer, 'result' is the human's answer and 'eval' indicates whether or not the human answer was correct or incorrect."
    #request="If all 'eval' values are 'CORRECT', return 'Congratulations! You're an expert :tada:'. Otherwise, I want you to explain to the human, refering to them in the second person, what are their main difficulties, based on their incorrect answers; don't give any concrete examples. Conversation:"
    #improve=convo.predict(input=intro+request+str(cyk_hist))
    
    return improve

In [835]:
panels = [] 


interactive_conversation = pn.bind(qa_evaluator, button_conversation)

conversation = pn.Column(
    pn.Row(ne_prompt, button_conversation),
    pn.Row(pn.pane.Markdown( "Aurora:", width=100)
           ,pn.pane.Markdown( f"""My name is {CHAT_NAME} and I’m about to check your knowledge! 👀""", width=600, height = 20)),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

# Study

In [836]:
def load_files():
    PATH = 'temp_files'
    FILE = 'Handbook.pdf'
    loader = PyPDFLoader(join(PATH,FILE))
    data_handbook = loader.load()
    
    text_splitter= CharacterTextSplitter(
        separator='\n',
        chunk_size=1000,
        chunk_overlap=150,
        length_function=len
    )
    data_handbook=text_splitter.split_documents(data_handbook)

    PATH = 'temp_files'
    FILE = 'Organizational Chart.json'

    # load JSON    
    chart_file = json.load(open(join(PATH,FILE)))
    chart_file=json.dumps(chart_file).split("key")
    
    embedding=OpenAIEmbeddings(openai_api_key=openai_ada_key
                                  , openai_api_base = openai_ada_base
                                  , deployment = openai_ada_deployment
                                  , openai_api_type = openai_api_type
                                  ,chunk_size=1)

    
    persist_directory='docs/chroma'
    vectordb_chart=Chroma.from_texts(
        texts=chart_file,
        embedding=embedding
    )
    
    persist_directory='docs/chroma'
    vectordb_handbook=Chroma.from_documents(
        documents=data_handbook,
        embedding=embedding
    )
    return vectordb_chart, vectordb_handbook

In [846]:
responses = []
memory_buddy = []
memory_network = []
def budy(ai, vectordb_handbook,vectordb_chart, buddy_prompt):
    global memory_buddy
    global memory_network    

    handbook_retriever = vectordb_handbook.as_retriever(search_type="similarity")
    chart_retriever = vectordb_chart.as_retriever(search_type="similarity")
    
    print(buddy_prompt[0:3])
    if 'who' in (buddy_prompt).lower():
        print('budy1_who')
        prompt_template = """Use the following pieces of context to answer the question at the end. If you don't
                            know the answer, just say that you don't know, don't try to make up an answer.
                            {context}
                            Question: {question}
                            """

        PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

        chain_sb = load_qa_chain(ai, chain_type="stuff", prompt=PROMPT, memory=memory_buddy,
                                 output_key="topic")

        chain_sb({"input_documents": chart_retriever.get_relevant_documents(buddy_prompt), "question": buddy_prompt},
                 return_only_outputs=True)

        print('budy2_who')    
        prompt_template = """Considering the following context:
        {context}
        
        Give me motivation me to go talk to the person:
        {question}
        
        If you don't know the answer, just wish me good luck, don't try to make up an answer.
        
    """

        PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

        chain_nw = load_qa_chain(ai, chain_type="stuff", prompt=PROMPT, memory=memory_network,
                                 output_key="people")
        chain_nw({"input_documents": chart_retriever.get_relevant_documents(buddy_prompt), "question":
                  memory_buddy.chat_memory.messages[-1].content}, return_only_outputs=True)
    else:  
        print('budy1')
        prompt_template = """Use the following pieces of context to answer the question at the end. If you don't
                            know the answer, just say that you don't know, don't try to make up an answer.
                            Return the answer together with the source (document and pages).
                            {context}
                            Question: {question}
                            """

        PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

        chain_sb = load_qa_chain(ai, chain_type="stuff", prompt=PROMPT, memory=memory_buddy,
                                 output_key="topic")

        chain_sb({"input_documents": handbook_retriever.get_relevant_documents(buddy_prompt), "question": buddy_prompt},
                 return_only_outputs=True)

        print('budy2')    
        prompt_template = """Given the following information regarding company workers, give up to three recommendations about the best person to talk about:
    Only recommend someone if you are confident they are a relevant person for the topic and if the topic is clear. Otherwise, don't recommend anyone.
    Context about company workers:{context}
    Topic: {question}
    """

        PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

        chain_nw = load_qa_chain(ai, chain_type="stuff", prompt=PROMPT, memory=memory_network,
                                 output_key="people")
        chain_nw({"input_documents": chart_retriever.get_relevant_documents(buddy_prompt), "question":
                  memory_buddy.chat_memory.messages[-1].content}, return_only_outputs=True)

In [847]:
counter_qa_respond = 0
vectordb_chart = []
vectordb_handbook = []
def qa_respond(_):
    global counter_qa_respond
    counter_qa_respond +=1
    global memory_buddy
    global memory_network
    global vectordb_handbook
    global vectordb_chart

    ai=AzureChatOpenAI(openai_api_base=AZURE_OPENAI_ENDPOINT
             ,openai_api_key=AZURE_OPENAI_KEY
             ,openai_api_type='azure'
             ,openai_api_version="2023-05-15"
             ,deployment_name =AZURE_ENGINE_NAME
             ,model = "gpt-3.5-turbo"
             ,temperature=0.1
        )
    
    print(str(button_query.clicks))
    # load PDF File
    if counter_qa_respond == 1:
        responses.append(
            pn.Row(
                pn.pane.Markdown( "Aurora:", width=100)
               ,pn.pane.Markdown( f"""My name is {CHAT_NAME} and I 
               can be your study buddy! 🫶""", width=600)))
    if counter_qa_respond == 2:
        print('buddy: ', len(vectordb_dict))
        if len(vectordb_dict)>=2:
            print('already had loaded documents')
            vectordb_chart = vectordb_dict['Organizational Chart'] 
            vectordb_handbook = vectordb_dict['Handbook']
        else:
            print("couldn't find loaded documents; loading...")
            pn.state.notifications.warning('Loading documentation (the first message takes a little bit longer!).', duration = 5000)
            vectordb_chart, vectordb_handbook = load_files()
            print('Loaded')
       
        memory_buddy = ConversationSummaryMemory(memory_key="chat_history",
                  llm=AzureChatOpenAI(openai_api_base=AZURE_OPENAI_ENDPOINT
                 ,openai_api_key=AZURE_OPENAI_KEY
                 ,openai_api_type='azure'
                 ,openai_api_version="2023-05-15"
                 ,deployment_name =AZURE_ENGINE_NAME
                 ,model = "gpt-3.5-turbo"
                 ,temperature=0.1
            ), return_messages=True, input_key="question", human_prefix='new employee', ai_prefix='Aurora')
    
        
        memory_network = ConversationSummaryMemory(memory_key="chat_history",
                 llm=AzureChatOpenAI(openai_api_base=AZURE_OPENAI_ENDPOINT
                 ,openai_api_key=AZURE_OPENAI_KEY
                 ,openai_api_type='azure'
                 ,openai_api_version="2023-05-15"
                 ,deployment_name =AZURE_ENGINE_NAME
                 ,model = "gpt-3.5-turbo"
                 ,temperature=0.1
            ), return_messages=True, input_key="question", human_prefix='new employee', ai_prefix='Aurora')
    
    
        buddy_prompt = ne_query.value
        ne_query.value = '' 
        budy(ai,vectordb_handbook,vectordb_chart, buddy_prompt)
        print('budy3')

        output1 = memory_buddy.chat_memory.messages[-1].content
        output2 = memory_network.chat_memory.messages[-1].content

        responses.append(
        pn.Row(pn.pane.Markdown( "New Employee:", width=100), pn.pane.Markdown(buddy_prompt, width=600)))
        responses.append(
        pn.Row(pn.pane.Markdown( "Aurora:", width=100), pn.pane.Markdown(output1, width=600)))
        responses.append(
        pn.Row(pn.pane.Markdown( "Aurora:", width=100), pn.pane.Markdown(output2, width=600)))
    if counter_qa_respond > 2:
        buddy_prompt = ne_query.value
        ne_query.value = '' 
        budy(ai,vectordb_handbook,vectordb_chart, buddy_prompt)
        print('budy3')

        output1 = memory_buddy.chat_memory.messages[-1].content
        output2 = memory_network.chat_memory.messages[-1].content

        responses.append(
        pn.Row(pn.pane.Markdown( "New Employee:", width=100), pn.pane.Markdown(buddy_prompt, width=600)))
        responses.append(
        pn.Row(pn.pane.Markdown( "Aurora:", width=100), pn.pane.Markdown(output1, width=600)))
        responses.append(
        pn.Row(pn.pane.Markdown( "Aurora:", width=100), pn.pane.Markdown(output2, width=600)))        
    return pn.Column(*responses, height = 300, scroll=True)

In [839]:
ne_query = pn.widgets.TextInput(value="", placeholder='Enter text here...', height = 50)
button_query = pn.widgets.Button(name="Talk!", width = 200, height = 40)


In [840]:
panels = [] 


interactive_buddy = pn.bind(qa_respond, button_query)

study = pn.Column(
    pn.Row(ne_query, button_query),
    pn.panel(interactive_buddy, height=300, loading_indicator=True),
)

0


# Status

In [841]:

def runner_hb(button_query):
    global pbar_hb
    if isinstance(memory_buddy, list): 
        v1=0 
    else: 
        v1=int(len(memory_buddy.chat_memory.messages)/2)
    yield pn.indicators.Progress(name='Progress', value=pbar_hb, width=200,max= 6, active=True, styles={'width': '200px', 'height': '20px'} )

#progress_handbook=pn.bind(runner_hb, button_query)
progress_handbook = pn.indicators.Progress(name='Progress', value=pbar_hb, width=200,max= 6, active=True, styles={'width': '200px', 'height': '20px'} )

def runner_role(button_query):
    global pbar_role
    if isinstance(memory_buddy, list): 
        v1=0 
    else: 
        v1=int(len(memory_buddy.chat_memory.messages)/2)
    yield pn.indicators.Progress(name='Progress', value=pbar_role, width=200,max= 6, active=True, styles={'width': '200px', 'height': '20px'})
    
#progress_role=pn.bind(runner_role, button_query)
progress_role = pn.indicators.Progress(name='Progress', value=pbar_role, width=200,max= 6, active=True, styles={'width': '200px', 'height': '20px'})

def runner_chart(button_query):
    global pbar_chart
    if isinstance(memory_buddy, list): 
        v1=0 
    else: 
        v1=int(len(memory_buddy.chat_memory.messages)/2)
    yield pn.indicators.Progress(name='Progress', value=pbar_chart, width=200, max= 6, active=True, styles={'width': '200px', 'height': '20px'})
    
# progress_chart=pn.bind(runner_chart, button_query)
progress_chart = pn.indicators.Progress(name='Progress', value=pbar_chart, width=200,max= 6, active=True, styles={'width': '200px', 'height': '20px'})

def pie_chart(button_query, button_conversation):
    global chart_hb
    global chart_role
    global chart_chart

    if isinstance(memory_buddy, list): 
        v1=0 
    else: 
        v1=len(memory_buddy.chat_memory.messages)/2

    
    print('\n\n charts:  ' + str(chart_hb)+ ' ' +  str(chart_role) + ' ' +  str(chart_chart) + ' ' + str(v1))
    
    x = {
        'Study Buddy': v1,
        'Handbook Knowledge': chart_hb,
        'Role specific Knowledge': chart_role,
        'Organizational Chart Knowledge': chart_chart,
    }

    data = pd.Series(x).reset_index(name='value').rename(columns={'index':'Tool'})
    data['angle'] = data['value']/data['value'].sum() * 2*pi
    data['color'] = Pastel1[len(x)]

    p = figure(height=400,width=700, title="", toolbar_location=None,
               tools="hover", tooltips="@Tool: @value", x_range=(-0.5, 1.0))

    r = p.wedge(x=0, y=1, radius=0.4,
            start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
            line_color="white", fill_color='color', legend_field='Tool', source=data)

    p.axis.axis_label=None
    p.axis.visible=False
    p.grid.grid_line_color = None



    bokeh_pane = pn.pane.Bokeh(p, theme="dark_minimal", height = 400)

    yield bokeh_pane
    
pie_yum=pn.bind(pie_chart, button_query, button_conversation)

In [842]:
def qa_memory(_):
    if isinstance(memory_buddy, list):
        out='Start talking to the Study buddy!'
    else:
        out=memory_buddy.buffer
    return out
mem_buddy = pn.bind(qa_memory, button_query)
def cyk_r_memory(_):
    if len(cyk_hist)==0:
        out='Check your knowledge with Aurora!'
    else:
        out=cyk_recap(cyk_hist)
    return out
def cyk_w_memory(_):
    if len(cyk_hist)==0:
        out='Check your knowledge with Aurora!'
    else:
        out=cyk_weak(cyk_hist)
    return out
mem_cyk = pn.bind(cyk_r_memory, button_conversation)
weak_cyk = pn.bind(cyk_w_memory, button_conversation)

# App

In [843]:
card_hr =  pn.Column(
            pn.pane.Markdown("""-----"""),
            pn.Card(introduction_qa
            , radio_group
            , generate_qa
            , output_qa
            , collapsed = True
            , title='Create Onboarding Plan', styles=dict(background='White'))
)
cards_ne =  pn.Column(
            pn.pane.Markdown("""-----"""),    
            pn.Card(pn.pane.Markdown('##Your Study Buddy Recap', renderer='markdown')
                    ,mem_buddy
                    ,pn.pane.Markdown('##Check Your Knowledge Recap', renderer='markdown')
                    ,pn.pane.Markdown('###Your learnings', renderer='markdown')
                    ,mem_cyk
                    ,pn.pane.Markdown('###Your difficulties', renderer='markdown')
                    ,weak_cyk
                    ,pn.pane.Markdown('##Your progress', renderer='markdown')
                    ,pn.Row(progress_handbook,pn.pane.HTML('Handbook progress', width=200, styles={'font-size': '100%'}))
                    ,pn.Row(progress_role,pn.pane.HTML('Role specific progress',  width=200, styles={'font-size': '100%'}))
                    ,pn.Row(progress_chart,pn.pane.HTML('Organizational chart progress', width=200, styles={'font-size': '100%'}))
                    ,pn.pane.Markdown('##Your activity', renderer='markdown')
                    ,pn.Row(pie_yum, width=700, height = 420)
                    , collapsed = True
                    , title='Check your Progress', styles=dict(background='White')),    
            pn.pane.Markdown("""-----"""),
            pn.Card( study
                    , collapsed = True
                    , scroll = True
                    , title='Your Buddy Aurora', styles=dict(background='White')),
            pn.pane.Markdown("""-----"""),
            pn.Card( conversation
                    , collapsed = True
                    , scroll = True
                    , title='Check your Knowledge', styles=dict(background='White'))
            )


pn.Column(
    pn.pane.Markdown("""# Aurora 🌌 - Your Onboarding App"""),
    pn.Tabs(('Human Resources', card_hr), ('New Employee', cards_ne))
).servable()



 charts:  0 0 0 0


Column(sizing_mode='stretch_width')
    [0] Markdown(str, sizing_mode='stretch_width')
    [1] Tabs(sizing_mode='stretch_width')
        [0] Column(sizing_mode='stretch_width')
            [0] Markdown(str, sizing_mode='stretch_width')
            [1] Card(collapsed=True, sizing_mode='stretch_width', styles={'background': 'White'}, title='Create Onboarding Plan')
                [0] Row(sizing_mode='stretch_width')
                    [0] Alert(str, sizing_mode='stretch_width')
                [1] RadioButtonGroup(button_type='success', name='Radio Button Group', options=['Handbook', 'Organization...], value='Handbook', width=400)
                [2] Row(width=1000, width_policy='fixed')
                    [0] Column(width=310)
                        [0] FileInput(height=40, sizing_mode='fixed', width=300)
                        [1] Button(height=40, name='Aurora, generate Q..., sizing_mode='fixed', styles={'background': 'White', ...}, width=300)
                    [1] Column(width=210)
                        [0] RadioButtonGroup(button_type='warning', height=90, name='Radio Button Group', options=['Faster Q&A', ...], orientation='vertical', sizing_mode='fixed', value='Faster Q&A', width=200)
                    [2] Column(width=710)
                        [0] Row(width=300)
                            [0] Button(button_type='primary', height=40, name='Aurora, generate r..., sizing_mode='fixed', width=300)
                            [1] TooltipIcon(value='Button will o..., width=10)
                        [1] TextInput(placeholder='Enter role here...', sizing_mode='stretch_width', styles={'width': '300px', ...})
                [3] WidgetBox(scroll=True, width=1100)
                    [0] ParamFunction(function, _pane=Column, defer_load=False, loading_indicator=True, sizing_mode='stretch_width')
                    [1] ParamFunction(function, _pane=Column, defer_load=False, loading_indicator=True, sizing_mode='stretch_width')
        [1] Column(sizing_mode='stretch_width')
            [0] Markdown(str, sizing_mode='stretch_width')
            [1] Card(collapsed=True, sizing_mode='stretch_width', styles={'background': 'White'}, title='Check your Progress')
                [0] Markdown(str, renderer='markdown', sizing_mode='stretch_width')
                [1] ParamFunction(function, _pane=Markdown, defer_load=False, sizing_mode='stretch_width')
                [2] Markdown(str, renderer='markdown', sizing_mode='stretch_width')
                [3] Markdown(str, renderer='markdown', sizing_mode='stretch_width')
                [4] ParamFunction(function, _pane=Markdown, defer_load=False, sizing_mode='stretch_width')
                [5] Markdown(str, renderer='markdown', sizing_mode='stretch_width')
                [6] ParamFunction(function, _pane=Markdown, defer_load=False, sizing_mode='stretch_width')
                [7] Markdown(str, renderer='markdown', sizing_mode='stretch_width')
                [8] Row(sizing_mode='stretch_width')
                    [0] Progress(max=6, styles={'width': '200px', ...}, value=0, width=200)
                    [1] HTML(str, styles={'font-size': '100%'}, width=200)
                [9] Row(sizing_mode='stretch_width')
                    [0] Progress(max=6, styles={'width': '200px', ...}, value=0, width=200)
                    [1] HTML(str, styles={'font-size': '100%'}, width=200)
                [10] Row(sizing_mode='stretch_width')
                    [0] Progress(max=6, styles={'width': '200px', ...}, value=0, width=200)
                    [1] HTML(str, styles={'font-size': '100%'}, width=200)
                [11] Markdown(str, renderer='markdown', sizing_mode='stretch_width')
                [12] Row(height=420, sizing_mode='fixed', width=700)
                    [0] ParamFunction(function, _pane=Bokeh, defer_load=False, sizing_mode='stretch_width')
            [2] Markdown(str, sizing_mode='stretch_width')
            [3] Card(collapsed=True, scroll=True, sizing_mode='stretch_wid

In [741]:
config = {"headerControls": {'maximize':'remove', 'minimize':'remove', 'smallify':'remove'}}
pn.layout.FloatPanel(pn.pane.Markdown('# You''ve completed your Onboarding! 🥳'),theme='info', config=config, name='🎉Congratulations', margin=20)


FloatPanel(config={'headerControls': {'maxim...}, margin=20, name='🎉Congratulations', objects=[Markdown(str, ...], sizing_mode='stretch_width', theme='info')